# Load R libraries

## spatialLIBD (1.19.11)

In [ ]:
library(spatialLIBD)
print(packageVersion("spatialLIBD"))

library(ggplot2)
library(Polychrome)
library(scales)
library(dplyr)
library(ggpubr)
library(RColorBrewer)
library(patchwork)

library(pheatmap)

# Load R functions

In [ ]:
source("functions.r")

## Color settings

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8, repr.plot.res = 100)
k9_colors <- Polychrome::palette36.colors(9)
names(k9_colors) <- seq(1, 9, 1)

# Data path

In [ ]:
dataPATH_traDEGs = "files/tables/"
dataPATH_trajectories = "data/inhouse/"

# Load data

## Visium

In [ ]:
spe <- fetch_data(type = "spatialDLPFC_Visium")
spe # 28_916 x 113_927

## traDEG modules

In [ ]:
modules <- list(
    AST_dev = as.character(read.csv(file.path(dataPATH_traDEGs, "AST_cluster_dev.csv"), header = F)$V1), 
    AST_dev_mat = as.character(read.csv(file.path(dataPATH_traDEGs, "AST_cluster_dev-mat.csv"), header = F)$V1),
    AST_PA_spec_whole = as.character(read.csv(file.path(dataPATH_traDEGs, "AST_cluster_PA-spec whole.csv"), header = F)$V1), 
    AST_mat_aging = as.character(read.csv(file.path(dataPATH_traDEGs, "AST_cluster_mat-aging.csv"), header = F)$V1),
    AST_FA_spec_mat_aging = as.character(read.csv(file.path(dataPATH_traDEGs, "AST_cluster_FA-spec mat-aging.csv"), header = F)$V1), 
    AST_aging = as.character(read.csv(file.path(dataPATH_traDEGs, "AST_cluster_aging.csv"), header = F)$V1),
    MICRO_dev = as.character(read.csv(file.path(dataPATH_traDEGs, "MICRO_cluster_dev.csv"), header = F)$V1), 
    MICRO_dev_mat = as.character(read.csv(file.path(dataPATH_traDEGs, "MICRO_cluster_dev-mat.csv"), header = F)$V1),
    MICRO_mat_aging = as.character(read.csv(file.path(dataPATH_traDEGs, "MICRO_cluster_mat-aging.csv"), header = F)$V1), 
    MICRO_aging = as.character(read.csv(file.path(dataPATH_traDEGs, "MICRO_cluster_aging.csv"), header = F)$V1),
    OLIGO_dev = as.character(read.csv(file.path(dataPATH_traDEGs, "OLIGO_cluster_dev.csv"), header = F)$V1), 
    OLIGO_mat_aging = as.character(read.csv(file.path(dataPATH_traDEGs, "OLIGO_cluster_mat-aging.csv"), header = F)$V1),
    EN_dev = as.character(read.csv(file.path(dataPATH_traDEGs, "EN_cluster_dev.csv"), header = F)$V1), 
    EN_Deep_layer_spec_dev_mat = as.character(read.csv(file.path(dataPATH_traDEGs, "EN_cluster_Deep-layer-spec dev-mat.csv"), header = F)$V1),
    EN_Upper_layer_spec_whole = as.character(read.csv(file.path(dataPATH_traDEGs, "EN_cluster_Upper-layer-spec whole.csv"), header = F)$V1), 
    EN_Deep_layer_spec_mat = as.character(read.csv(file.path(dataPATH_traDEGs, "EN_cluster_Deep-layer-spec mat.csv"), header = F)$V1),
    EN_L5_6_NP_spec_mat_aging = as.character(read.csv(file.path(dataPATH_traDEGs, "EN_cluster_L5_6_NP-spec mat-aging.csv"), header = F)$V1), 
    EN_Deep_non_IT_spec_mat_aging = as.character(read.csv(file.path(dataPATH_traDEGs, "EN_cluster_Deep-non-IT-spec mat-aging.csv"), header = F)$V1),
    EN_Upper_layer_spec_mat_aging = as.character(read.csv(file.path(dataPATH_traDEGs, "EN_cluster_Upper-layer-spec mat-aging.csv"), header = F)$V1), 
    EN_mat_aging = as.character(read.csv(file.path(dataPATH_traDEGs, "EN_cluster_mat-aging.csv"), header = F)$V1),
    IN_CGE_spec_dev = as.character(read.csv(file.path(dataPATH_traDEGs, "IN_cluster_CGE-spec dev.csv"), header = F)$V1), 
    IN_dev = as.character(read.csv(file.path(dataPATH_traDEGs, "IN_cluster_dev.csv"), header = F)$V1),
    IN_MGE_spec_dev = as.character(read.csv(file.path(dataPATH_traDEGs, "IN_cluster_MGE-spec dev.csv"), header = F)$V1), 
    IN_sporadic_mat_aging = as.character(read.csv(file.path(dataPATH_traDEGs, "IN_cluster_sporadic mat-aging.csv"), header = F)$V1),
    IN_PVALB_CHC_spec_mat_aging = as.character(read.csv(file.path(dataPATH_traDEGs, "IN_cluster_PVALB_CHC-spec mat-aging.csv"), header = F)$V1), 
    IN_LAMP5_spec_mat_aging = as.character(read.csv(file.path(dataPATH_traDEGs, "IN_cluster_LAMP5-spec mat-aging.csv"), header = F)$V1),
    IN_ADARB2_spec_mat_aging = as.character(read.csv(file.path(dataPATH_traDEGs, "IN_cluster_ADARB2-spec mat-aging.csv"), header = F)$V1), 
    IN_mat_aging = as.character(read.csv(file.path(dataPATH_traDEGs, "IN_cluster_mat-aging.csv"), header = F)$V1)
)

# Calculate average PC1

## Calculate average PC1 (per spot)

In [ ]:
avg_PC1_per_spot_list <- lapply(modules, avg_PC1_per_spot)

## Calculate average PC1 (per cluster)

In [ ]:
# Generate based on per spot
avg_pc1 <- lapply(avg_PC1_per_spot_list, function(df) {
    df_avg <- df %>%
        group_by(BayesSpace_harmony_09, Sample_ID) %>%
        summarise(PC1_avg = mean(PC1, na.rm = TRUE), .groups = "drop")
    
    df_avg_cluster <- df_avg %>%
        group_by(BayesSpace_harmony_09) %>%
        summarise(PC1_avg_cluster = mean(PC1_avg, na.rm = TRUE), .groups = "drop")
    return(df_avg_cluster$PC1_avg_cluster)
})
avg_pc1

## Coverage matrix

In [ ]:
nonan <- as.data.frame(lapply(avg_PC1_per_spot_list, function(x){
    res <- as.vector(table(na.omit(x)$BayesSpace_harmony_09))
    return(res)
}))
nonan

## Z-score per module

In [ ]:
avg_pc1_zscore <- lapply(avg_pc1, scale)
avg_pc1_zscore

In [ ]:
avg_pc1_zscore_df <- as.data.frame(avg_pc1_zscore)
rownames(avg_pc1_zscore_df) <- c("Sp09D01 ~ L1", "Sp09D02 ~ L1", "Sp09D03 ~ L2", "Sp09D05 ~ L3", "Sp09D08 ~ L4", "Sp09D04 ~ L5", "Sp09D07 ~ L6", "Sp09D06 ~ WM", "Sp09D09 ~ WM")
avg_pc1_zscore_df

## Square heatmap

In [ ]:
pdf("figures/Visium_Maynard/Square_Heatmap_Cluster_Z_score_per_Module.pdf", width = 12, height = 8)
main_plot <- square_heatmap(as.data.frame(avg_pc1_zscore_df[c("Sp09D01 ~ L1", "Sp09D02 ~ L1", "Sp09D03 ~ L2", "Sp09D05 ~ L3", "Sp09D08 ~ L4", "Sp09D04 ~ L5", "Sp09D07 ~ L6", "Sp09D09 ~ WM", "Sp09D06 ~ WM"), ]), 
               nonan, 
               limits = c(-2, 2), 
               size_range = c(0.2, 0.8),
               row_labels = c("Sp09D01 ~ L1", "Sp09D02 ~ L1", "Sp09D03 ~ L2", "Sp09D05 ~ L3", "Sp09D08 ~ L4", "Sp09D04 ~ L5", "Sp09D07 ~ L6", "Sp09D09 ~ WM", "Sp09D06 ~ WM"), 
               col_labels = names(modules))

size_leg <- square_size_legend(breaks = rev(c(min(nonan), 800, 8000, 16000, max(nonan))), size_range = c(0.2, 0.8))

main_plot + size_leg + plot_layout(widths = c(1, 0.15))

dev.off()

# Spatial domain visualization

In [ ]:
sub_sample <- "Br2743_mid"

pdf(paste0("figures//Visium_Maynard/BayersSpace_cluster_9_", sub_sample,".pdf"), width = 5.5, height = 5.5)
# Extract Age information
age_value <- unique(colData(spe)[colData(spe)$sample_id == sub_sample, "age"])

# Generate plot
p09_sub <- spatialLIBD::vis_clus(
    spe = spe,
    clustervar = "BayesSpace_harmony_09",
    sampleid = sub_sample,
    point_size = 1.5,
    colors = k9_colors
)

# Add Age information to title
p09_sub <- p09_sub + ggtitle(paste0(sub_sample, ": ", age_value))

# Display plot
print(p09_sub)
dev.off()



# Multiple genes visualization

## Vis_gene in sample Br2743_mid

### AST

In [ ]:
sub_module <- "AST_PA_spec_whole"
genes <- rowData(spe)$gene_search[rowData(spe)$gene_name %in% modules[[sub_module]]]
pdf(paste0("figures//Visium_Maynard/vis_gene_", sub_module, "_", sub_sample,"_pca.pdf"), width = 5.5, height = 5.5)
# Extract Age information
age_value <- unique(colData(spe)[colData(spe)$sample_id == sub_sample, "age"])

# Generate plot
p09_sub <- vis_gene(
    spe = spe,
    geneid = genes,
    sampleid = sub_sample,
    multi_gene_method = "pca",
    point_size = 1.5, limits = c(0, 4)
) + ggtitle(paste0(sub_sample, ": ", age_value, " ( ", sub_module, " ", length(genes), " )"))

# Display plot
print(p09_sub)
dev.off()

sub_module <- "AST_FA_spec_mat_aging"
genes <- rowData(spe)$gene_search[rowData(spe)$gene_name %in% modules[[sub_module]]]
pdf(paste0("figures//Visium_Maynard/vis_gene_", sub_module, "_", sub_sample,"_pca.pdf"), width = 5.5, height = 5.5)
# Extract Age information
age_value <- unique(colData(spe)[colData(spe)$sample_id == sub_sample, "age"])

# Generate plot
p09_sub <- vis_gene(
    spe = spe,
    geneid = genes,
    sampleid = sub_sample,
    multi_gene_method = "pca",
    point_size = 1.5, limits = c(0, 6)
) + ggtitle(paste0(sub_sample, ": ", age_value, " ( ", sub_module, " ", length(genes), " )"))

# Display plot
print(p09_sub)
dev.off()


### MICRO

In [ ]:
sub_module <- "MICRO_dev"
genes <- rowData(spe)$gene_search[rowData(spe)$gene_name %in% modules[[sub_module]]]
pdf(paste0("figures//Visium_Maynard/vis_gene_", sub_module, "_", sub_sample,"_pca.pdf"), width = 5.5, height = 5.5)
# Extract Age information
age_value <- unique(colData(spe)[colData(spe)$sample_id == sub_sample, "age"])

# Generate plot
p09_sub <- vis_gene(
    spe = spe,
    geneid = genes,
    sampleid = sub_sample,
    multi_gene_method = "pca",
    point_size = 1.5, limits = c(0, 4)
) + ggtitle(paste0(sub_sample, ": ", age_value, " ( ", sub_module, " ", length(genes), " )"))

# Display plot
print(p09_sub)
dev.off()

sub_module <- "MICRO_aging"
genes <- rowData(spe)$gene_search[rowData(spe)$gene_name %in% modules[[sub_module]]]
pdf(paste0("figures//Visium_Maynard/vis_gene_", sub_module, "_", sub_sample,"_pca.pdf"), width = 5.5, height = 5.5)
# Extract Age information
age_value <- unique(colData(spe)[colData(spe)$sample_id == sub_sample, "age"])

# Generate plot
p09_sub <- vis_gene(
    spe = spe,
    geneid = genes,
    sampleid = sub_sample,
    multi_gene_method = "pca",
    point_size = 1.5, limits = c(0, 10)
) + ggtitle(paste0(sub_sample, ": ", age_value, " ( ", sub_module, " ", length(genes), " )"))

# Display plot
print(p09_sub)
dev.off()


### OLIGO

In [ ]:
sub_module <- "OLIGO_dev"
genes <- rowData(spe)$gene_search[rowData(spe)$gene_name %in% modules[[sub_module]]]
pdf(paste0("figures//Visium_Maynard/vis_gene_", sub_module, "_", sub_sample,"_pca.pdf"), width = 5.5, height = 5.5)
# Extract Age information
age_value <- unique(colData(spe)[colData(spe)$sample_id == sub_sample, "age"])

# Generate plot
p09_sub <- vis_gene(
    spe = spe,
    geneid = genes,
    sampleid = sub_sample,
    multi_gene_method = "pca",
    point_size = 1.5, limits = c(0, 5)
) + ggtitle(paste0(sub_sample, ": ", age_value, " ( ", sub_module, " ", length(genes), " )"))

# Display plot
print(p09_sub)
dev.off()

sub_module <- "OLIGO_mat_aging"
genes <- rowData(spe)$gene_search[rowData(spe)$gene_name %in% modules[[sub_module]]]
pdf(paste0("figures//Visium_Maynard/vis_gene_", sub_module, "_", sub_sample,"_pca.pdf"), width = 5.5, height = 5.5)
# Extract Age information
age_value <- unique(colData(spe)[colData(spe)$sample_id == sub_sample, "age"])

# Generate plot
p09_sub <- vis_gene(
    spe = spe,
    geneid = genes,
    sampleid = sub_sample,
    multi_gene_method = "pca",
    point_size = 1.5, limits = c(0, 15)
) + ggtitle(paste0(sub_sample, ": ", age_value, " ( ", sub_module, " ", length(genes), " )"))

# Display plot
print(p09_sub)
dev.off()


### EN

In [ ]:
sub_module <- "EN_Deep_non_IT_spec_mat_aging"
genes <- rowData(spe)$gene_search[rowData(spe)$gene_name %in% modules[[sub_module]]]
pdf(paste0("figures//Visium_Maynard/vis_gene_", sub_module, "_", sub_sample,"_pca.pdf"), width = 5.5, height = 5.5)
# Extract Age information
age_value <- unique(colData(spe)[colData(spe)$sample_id == sub_sample, "age"])

# Generate plot
p09_sub <- vis_gene(
    spe = spe,
    geneid = genes,
    sampleid = sub_sample,
    multi_gene_method = "pca",
    point_size = 1.5, limits = c(0, 3)
) + ggtitle(paste0(sub_sample, ": ", age_value, " ( ", sub_module, " ", length(genes), " )"))

# Display plot
print(p09_sub)
dev.off()

sub_module <- "EN_Upper_IT_spec_mat_aging"
genes <- rowData(spe)$gene_search[rowData(spe)$gene_name %in% modules[[sub_module]]]
pdf(paste0("figures//Visium_Maynard/vis_gene_", sub_module, "_", sub_sample,"_pca.pdf"), width = 5.5, height = 5.5)
# Extract Age information
age_value <- unique(colData(spe)[colData(spe)$sample_id == sub_sample, "age"])

# Generate plot
p09_sub <- vis_gene(
    spe = spe,
    geneid = genes,
    sampleid = sub_sample,
    multi_gene_method = "pca",
    point_size = 1.5, limits = c(0, 3)
) + ggtitle(paste0(sub_sample, ": ", age_value, " ( ", sub_module, " ", length(genes), " )"))

# Display plot
print(p09_sub)
dev.off()


## Violin plot in all 30 samples

In [ ]:
for(module in names(avg_PC1_per_spot_list)){
    df_avg <- avg_PC1_per_spot_list[[module]] %>%
        group_by(BayesSpace_harmony_09, Sample_ID) %>%
        summarise(PC1_avg = mean(PC1, na.rm = TRUE), .groups = "drop")

    pdf(paste0("figures/Visium_Maynard/violin_per_cluster_PC1_traDEG_modules/", module,".pdf"), width = 12, height = 6)
    p <- ggplot(df_avg, aes(x = BayesSpace_harmony_09, y = PC1_avg, fill = BayesSpace_harmony_09)) +
        geom_violin() +
        geom_boxplot(width = 0.1, outlier.shape = NA) +
        geom_jitter(shape = 16, position = position_jitter(0.2)) +
        theme_minimal() +
        labs(title = paste0("PC1 Distribution by Sp09 Cluster (", module,")"), x = "Sp09 Cluster", y = "PC1") + 
        scale_fill_manual(values = k9_colors)
    print(p)
    dev.off()
}